# Dataset Benchmarking

This notebook is a behind-the-scenes benchmarking notebook, mainly for use by developers. The recommended way for users to interact with the dataset is via the `Measurement` object and its associated context manager. See the corresponding notebook for a comprehensive toturial on how to use those.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import qcodes as qc
from qcodes import ParamSpec, new_data_set, new_experiment, initialise_database, load_or_create_experiment
import numpy as np

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : /home/vsts/.qcodes/logs/command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active


Qcodes Logfile : /home/vsts/.qcodes/logs/200926-18145-qcodes.log


In [2]:
qc.config.core.db_location

'~/experiments.db'

In [3]:
initialise_database()

## Setup

In [4]:
exp = load_or_create_experiment("benchmarking", sample_name="the sample is a lie")
exp

benchmarking#the sample is a lie#1@/home/vsts/experiments.db
------------------------------------------------------------

Now we can create a dataset. Note two things:

    - if we don't specfiy a exp_id, but we have an experiment in the experiment container the dataset will go into that one.
    - dataset can be created from the experiment object
    

In [5]:
dataSet = new_data_set("benchmark_data", exp_id=exp.exp_id)
exp

benchmarking#the sample is a lie#1@/home/vsts/experiments.db
------------------------------------------------------------
1-benchmark_data-1--0

In this benchmark we will assueme that we are doing a 2D loop and investigate the performance implications of writing to the dataset

In [6]:
x_shape = 100
y_shape = 100

## Baseline: Generate data

In [7]:
%%time
for x in range(x_shape):
    for y in range(y_shape):
        z = np.random.random_sample(1)

CPU times: user 16.3 ms, sys: 33 µs, total: 16.4 ms
Wall time: 16.1 ms


and store in memory

In [8]:
x_data = np.zeros((x_shape, y_shape))
y_data = np.zeros((x_shape, y_shape))
z_data = np.zeros((x_shape, y_shape))

In [9]:
%%time
for x in range(x_shape):
    for y in range(y_shape):
        x_data[x,y] = x
        y_data[x,y] = y
        z_data[x,y] = np.random.random_sample()

CPU times: user 9.57 ms, sys: 0 ns, total: 9.57 ms
Wall time: 9.46 ms


## Add to dataset inside double loop

In [10]:
double_dataset = new_data_set("doubledata", exp_id=exp.exp_id,
                              specs=[ParamSpec("x", "numeric"),
                                     ParamSpec("y", "numeric"),
                                     ParamSpec('z', "numeric")])
double_dataset.mark_started()

Note that this is so slow that we are only doing a 10th of the computation

In [11]:
%%time
for x in range(x_shape//10):
    for y in range(y_shape):
        double_dataset.add_result({"x": x, 'y': y, 'z': np.random.random_sample()})

2020-09-26 12:36:34,419 ¦ py.warnings ¦ WARNING ¦ warnings ¦ _showwarnmsg ¦ 110 ¦ /home/vsts/work/1/s/qcodes/utils/deprecate.py:58: QCoDeSDeprecationWarning: The function <add_result> is deprecated. Use "add_results" as an alternative.
  issue_deprecation_warning(f'{t} <{n}>', reason, alternative)



CPU times: user 332 ms, sys: 83 ms, total: 415 ms
Wall time: 1.18 s


## Add the data in outer loop and store as np array

In [12]:
single_dataset = new_data_set("singledata", exp_id=exp.exp_id,
                              specs=[ParamSpec("x", "array"),
                                     ParamSpec("y", "array"),
                                     ParamSpec('z', "array")])
single_dataset.mark_started()
x_data = np.zeros((y_shape))
y_data = np.zeros((y_shape))
z_data = np.zeros((y_shape))

In [13]:
%%time
for x in range(x_shape):
    for y in range(y_shape):
        x_data[y] = x
        y_data[y] = y
        z_data[y] = np.random.random_sample(1)
    single_dataset.add_result({"x": x_data, 'y': y_data, 'z': z_data})

CPU times: user 104 ms, sys: 3.09 ms, total: 107 ms
Wall time: 129 ms


## Save once after loop

In [14]:
zero_dataset = new_data_set("zerodata", exp_id=exp.exp_id,
                            specs=[ParamSpec("x", "array"),
                                   ParamSpec("y", "array"),
                                   ParamSpec('z', "array")])
zero_dataset.mark_started()
x_data = np.zeros((x_shape, y_shape))
y_data = np.zeros((x_shape, y_shape))
z_data = np.zeros((x_shape, y_shape))

In [15]:
%%time
for x in range(x_shape):
    for y in range(y_shape):
        x_data[x,y] = x
        y_data[x,y] = y
        z_data[x,y] = np.random.random_sample(1)
zero_dataset.add_result({'x':x_data, 'y':y_data, 'z':z_data})

CPU times: user 22.7 ms, sys: 361 µs, total: 23.1 ms
Wall time: 24 ms


1

## Array parameter

In [16]:
array1D_dataset = new_data_set("array1Ddata", exp_id=exp.exp_id,
                               specs=[ParamSpec("x", "array"),
                                      ParamSpec("y", "array"),
                                      ParamSpec('z', "array")])
array1D_dataset.mark_started()
y_setpoints = np.arange(y_shape)

In [17]:
%%timeit
for x in range(x_shape):
    x_data[x,:] = x
    array1D_dataset.add_result({'x':x_data[x,:], 'y':y_setpoints, 'z':np.random.random_sample(y_shape)})

113 ms ± 1.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
x_data = np.zeros((x_shape, y_shape))
y_data = np.zeros((x_shape, y_shape))
z_data = np.zeros((x_shape, y_shape))
y_setpoints = np.arange(y_shape)

In [19]:
array0D_dataset = new_data_set("array0Ddata", exp_id=exp.exp_id,
                               specs=[ParamSpec("x", "array"),
                                      ParamSpec("y", "array"),
                                      ParamSpec('z', "array")])
array0D_dataset.mark_started()

In [20]:
%%timeit
for x in range(x_shape):
    x_data[x,:] = x
    y_data[x,:] = y_setpoints
    z_data[x,:] = np.random.random_sample(y_shape)
array0D_dataset.add_result({'x':x_data, 'y':y_data, 'z':z_data})

6.94 ms ± 118 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Insert many

In [21]:
data = []
for i in range(100):
    for j in range(100):
        data.append({'x': i, 'y':j, 'z':np.random.random_sample()})

In [22]:
many_Data = new_data_set("many_data", exp_id=exp.exp_id,
                         specs=[ParamSpec("x", "numeric"),
                                ParamSpec("y", "numeric"),
                                ParamSpec("z", "numeric")])
many_Data.mark_started()

In [23]:
%%timeit
many_Data.add_results(data)

36.5 ms ± 2.31 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
